# Variational Autoencoder Neural Network, MNIST

In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time

In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

from torch.autograd import Variable
from torchvision.utils import save_image

---